# Tanya Tandon Week 3 Homework

QUESTION 1

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pprint import pprint
import requests
import datetime
import time
import statistics

places = ['New Delhi', 'Evanston', 'LA', 'Paris', 'London']
latitude = [28.644800,42.045597,34.052235, 48.864716, 51.508530]
longitude = [77.216721,-87.688568, -118.243683, 2.349014, -0.076132]
unix_ts = []
for i in range(30):
    dt = datetime.datetime(2018, 1, (i+1), 1, 0)
    unix_ts.append(int(time.mktime(dt.timetuple())))

min_f =[]
max_f =[]
mid_f=[]
for p in range(len(places)):
    j = 0
    min_d =[]
    max_d =[]
    mid_d =[]
    for i in unix_ts: 
        
        response = requests.get("https://api.darksky.net/forecast/b3fb7c92f98f5a42384809b8ef1feae3/{},{},{}".format(latitude[p], longitude[p], i))           
        result = response.json()
        
        min_d.append(result['daily']['data'][0]['temperatureMin'])
        max_d.append(result['daily']['data'][0]['temperatureHigh'])
        mid_d.append((min_d[j] + max_d[j])/2)
        j = j +1
    min_f.append(min(min_d))
    max_f.append(max(max_d))
    mid_f.append(statistics.mean(mid_d))

print("Min" + "         " + "Max" + "         " + "Mid" )
for i in range(5):
    print(str(min_f[i]) + "       " + str(max_f[i])+ "       "+ str(mid_f[i]))
plt.plot(places, min_f, color= 'b', label = 'min')
plt.plot(places, max_f, color = 'r', label = 'max')
plt.plot(places, mid_f, color = 'm', label = 'mid')
plt.legend()
plt.show()

QUESTION 2

In [ ]:
from requests import get 
from bs4 import BeautifulSoup

response_2 = get("https://report.boonecountymo.org/mrcjava/servlet/SH01_MP.I00290s?max_rows=500" )
html = response_2.content.decode('ascii')

#html = open("Main Page.rtf").read()

html_soup = BeautifulSoup(html, 'html.parser')
change_no = html_soup.find_all('input', class_ = "mrcinput")

classcon = html_soup.find_all('tr', class_=["even", "odd"])

last_t = []
first_t =[]
age_t = []
n_l = []
for i in range(len(classcon)- 1):
    string1 = classcon[i+1].get_text().split('\n')
    last_t.append(string1[1]) 
    first_t.append(string1[2])
    age_t.append(string1[6])
    n_l.append("https://report.boonecountymo.org/mrcjava/servlet/" + classcon[i+1].find("a")["href"]) 


In [ ]:

# SEQUENTIAL
def sequential():
    total_cases = []
    total_bail = []
    for i in range(len(classcon)- 1):  
        response3 = get(n_l[i])
        htmln1 = response3.content.decode('ascii')
        new_soup1 = BeautifulSoup(htmln1,'html.parser')
        new_con1 = new_soup1.find_all('tr', class_= ["detailBackground"])
        new_total1 = new_soup1.find_all('tr', class_="grandBackground")

        if(len(new_con1) != 0): 
            total_bail.append(new_total1[0].get_text().split('\n')[4])
        else:
            total_bail.append("0")
        total_cases.append(len(new_con1))
    for i in range(len(classcon)- 1):
        if (total_bail[i] == ".00"):
            total_bail[i] = 0
        print( str(last_t[i]) + " "  + str(first_t[i]) + "               "  + str(total_cases[i]) +  "          " + str(total_bail[i]))

In [ ]:
#THREADING
def threading():
    import threading

    def print_it(n_t,i):    

        response3 = get(n_t)
        htmln1 = response3.content.decode('ascii')
        #htmlnt = open("Detail Page.rtf".read)
        new_soup1 = BeautifulSoup(htmln1,'html.parser')
        new_con1 = new_soup1.find_all('tr', class_= ["detailBackground"])
        new_total1 = new_soup1.find_all('tr', class_="grandBackground")
        tLock.acquire()
        if new_con1: 
            print(str(last_t[i]) + "  " + str(first_t[i]) + "               "  + str(new_total1[0].get_text().split('\n')[4]) +  "          " + str(len(new_con1)))
        else:
            print(str(last_t[i]) + "  " + str(first_t[i]) + "               "  + str("0") +  "          " + str(len(new_con1)))
        tLock.release()

    tLock = threading.Lock()

    for j in range(len(classcon)- 1): 
        t1 = Thread(target= print_it, args=(n_l[j],j))
        t1.start()



In [ ]:
import timeit
t1 = timeit.timeit(sequential)
t2 = timeit.timeit(threading)
print( "Time for sequential:  " +  str(t1))
print( "Time for threading:  " + str(t2))
